In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from pandas import DataFrame
import datetime as dt
#pulling data API

key =credentials.login['key_isaham']  # extract password

### Function Training

In [2]:
technology_training =['AMTEL',
                    'APPASIA',
                    'ARBB',
                    'D&O',
                    'DNEX',
                    'DNONCE',
                    'DPHARMA',
                    'DUFU',
                    'EFORCE',
                    'ELSOFT',
                    'FPGROUP',
                    'FRONTKN',
                    'GHLSYS',
                    'GLOTEC',
                    'GPACKET', 
                    'GREATEC',
                    'GTRONIC',
                    'IFCAMSC',
                    'INARI',
                    'INNITY', 
                    'INSAS',
                    'JFTECH',
                    'JHM',
                    'KRONO',
                    'KUB',
                    'MI',
                    'MICROLN',
                    'MMSV',
                    'MPI',
                    'MYEG',
                    'N2N',
                    'OPENSYS',
                    'PENTA',
                    'REXIT', 
                    'SCICOM', 
                    'SCOMNET',
                    'THETA',
                    'TURIYA', 
                    'UNISEM',
                    'UWC',
                    'VIS',
                    'VITROX',
                    'VIVOCOM',
                    'WILLOW']

        

In [3]:
def training_data(technology_training):
    ss={}
    d = {}


    for m,stock in enumerate (technology_training):

        print("----------------PROCESSING {}------------------".format(stock))
        symbol = stock    # Which stock to pull    
        response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
        data = response.json()

        # Tuning OB and OS level
        ob_cross=85
        os_cross=35

        df= pd.DataFrame.from_dict(data)
        df= df[['c','d','h','l','o','v']]
        df['d'] = pd.to_datetime(df['d'], dayfirst= True )

        # Identify MA50
        n=50
        ma50=[] 

        for i in range(len(df["c"])-(n-1)):
            ma = round(sum(df["c"][i:i+(n)])/n,4)
            ma50.append(ma)

        df["MA50"] = pd.Series(ma50)
        df["MA50"] = df["MA50"].shift(n-1)

        # Filter uptrend price above ma50
        # Calculating the gap of both MA
        df["MAgap"] = df["c"] -  df["MA50"] 

        # Set date as index
        df.set_index('d',inplace=True,drop=False)
        df.index.name = None

        # Create signal for uptrend price above ma50 
        # Result not accurate for stochastic when just filter uptrend only
        df["Uptrend"]=df['MAgap']>0
        Uptrend=df["Uptrend"]
        Uptrend_signal=Uptrend[Uptrend==True]

        # Signal stochastic
        # Create the "L14" column in the DataFrame
        df['L14'] = df['l'].rolling(window=14).min()

        # Create the "H14" column in the DataFrame
        df['H14'] = df['h'].rolling(window=14).max()

        # Create the "%K" column in the DataFrame
        df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

        # Create the "%D" column in the DataFrame
        df['%D'] = df['%K'].rolling(window=3).mean()

        # Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
        # A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
        df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

        # Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
        # A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
        df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

        # Create a column in the DataFrame for name of stock
        df['Stock_Name'] = symbol

        # Define overbought and oversold
        overbought =df['Sell OB']
        oversold =df['Buy OS']

        # Use index(date) for create strategy using overbought and oversold
        date_os=list(oversold[oversold == True].index)
        date_ob=list(overbought[overbought == True].index)
        
        # Transaction recording
        buy = []
        sell = []
        sdate = []
        bdate = []
        wr = []
        pct = []
        vol = []
        ope =[]
        high = []
        low = []

        # Record close,open,vol,high,low for buy transaction
        for n,i in enumerate(date_os):
            buy.append(df['c'].loc[i])
            vol.append(df['v'].loc[i])
            ope.append(df['o'].loc[i])
            high.append(df['h'].loc[i])
            low.append(df['l'].loc[i])
            bdate.append(i)
            r=[]

            for m,k in enumerate(date_ob):

                if k>i:
                    r.append(m)
                    if len(r) ==1:
                        # Record close,percentage change for sell transaction
                        sell.append(df['c'].loc[k])
                        sdate.append(k)
                        pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                        pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                    elif len(r)>1:
                        pass
        diff =len(buy)-len(sell)
        # Checking for final transaction, make sure it is sell, as we want to close the position
        if len(buy) == len(sell):
            pass   
        elif len(buy) != len(sell):
            # Deleting excess buy transaction after we close position
            del(buy[-diff:])
            del(bdate[-diff:])
            del(ope[-diff:])
            del(high[-diff:])
            del(vol[-diff:])
            del(low[-diff:])
        dct = {
            "Buy_Date": bdate,
            "Buy": buy,
            "Sell": sell,
            "Sell_Date": sdate,
            "Open":ope,
            'Volume': vol,
            "Low": low,
            "high": high,
            "Pct_change": pct    
        }  
        # Save document for stochastic strategy
        trans = pd.DataFrame(dct)

        # Define variable for winning rate
        wr = []
        for i in range(len(trans)):
            if trans["Pct_change"][i] > 0:
                wr.append(1)
            else:
                wr.append(0)

        # Define variable for name of stock
        Name_stock = []
        for s in range(len(trans)):
            Name_stock.append(symbol)

        # Add new column
        trans["wr"] = pd.Series(wr)
        trans['Stock'] = pd.Series(Name_stock)

        # Scaling certain features only for training data
        col_names = ['Buy', 'Low', 'Open', 'Volume','high']  
        features = trans[col_names]

        # Prepare prediction data that no need to scale
        scaler = StandardScaler().fit(features.values)
        features = scaler.transform(features.values)

        # Convert the array back to a dataframe
        dataset = DataFrame(features)    
        col_notscale = ['Buy_Date','Stock','wr']
        features_notscale = trans[col_notscale]

        # Saving in multiple variable
        d["{}".format(stock)] = dataset
        ss["{}".format(stock)] = features_notscale
        print("----------------DONE {}------------------".format(symbol))

    

    # Combining all training data generated
    df_notscale = pd.concat(list(ss.values()),ignore_index=True)
    df_scale = pd.concat(list(d.values()),ignore_index=True)
    df_scale=df_scale.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})
    result_scaler = pd.concat([df_notscale, df_scale], axis=1)

    # Setting Dates as index
    df_all = result_scaler.sort_values(by="Buy_Date")
    df_all.set_index('Buy_Date', inplace=True)

    # Encoding the stock names
    le = LabelEncoder()
    df_all["Stock"] = le.fit_transform(df_all["Stock"])

    # Define the splitting date (for training we use 5 years data)
    # Splitting train test
    day = dt.datetime.today().day
    month = dt.datetime.today().month
    year = dt.datetime.today().year - 5
    date1 = "{}-{}-01".format(year, str(month).zfill(2), str(day).zfill(2))
    date2 = str(dt.datetime.today().date() - dt.timedelta(days=1))  # until yesterday only

    # Separate input features and target
    X = df_all.drop(['wr'], axis=1)
    y = df_all.wr

    # Splitting train test
    # train data (from listing IPO until 1 jan 2021)
    X_train, y_train = X['01-01-2016': '01-01-2021'], y['01-01-2016':'01-01-2021']

    X_trainss, y_train = X_train.to_numpy(), y_train.to_numpy()
    # Training the model
    upsampled = XGBClassifier(verbosity=0,random_seed=0).fit(X_trainss, y_train)
    return upsampled

model=training_data(technology_training)


----------------PROCESSING AMTEL------------------
----------------DONE AMTEL------------------
----------------PROCESSING APPASIA------------------
----------------DONE APPASIA------------------
----------------PROCESSING ARBB------------------
----------------DONE ARBB------------------
----------------PROCESSING D&O------------------
----------------DONE D&O------------------
----------------PROCESSING DNEX------------------
----------------DONE DNEX------------------
----------------PROCESSING DNONCE------------------
----------------DONE DNONCE------------------
----------------PROCESSING DPHARMA------------------
----------------DONE DPHARMA------------------
----------------PROCESSING DUFU------------------
----------------DONE DUFU------------------
----------------PROCESSING EFORCE------------------
----------------DONE EFORCE------------------
----------------PROCESSING ELSOFT------------------
----------------DONE ELSOFT------------------
----------------PROCESSING FPGROUP--

C:\Users\Public\Documents\anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [4]:
# filter by sector

In [5]:
def sector_filter(sector):

  # pulling data from api
  URL = 'https://admin.isaham.my/api/stocks/list'
  r = requests.get(url = URL) 
  data = r.json() 
  df = pd.DataFrame(data['stocks'],columns = ['code','symbol','name','primary-sector','secondary-sector'])

  wl = df[df["primary-sector"]==sector]
  wl = list(wl.symbol)

  return(wl)

In [6]:
sector = "Technology"
# defining the password
#key = credentials.login['key_isaham']
wl = sector_filter(sector)

### Function Prediction

In [7]:
# Define the instruments to download.


In [8]:
def prediction_data(wl):
    # looping through all stocks
    # Scale dictionary of dataframes for prediction set(scaling by stock)
    sp={}
    nsp={}

    for m,stock in enumerate (wl):

        print("----------------PROCESSING {}------------------".format(stock))
        symbol = stock    # Which stock to pull    
        response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
        data = response.json()

        # Tuning OB and OS level
        ob_cross=85
        os_cross=35

        df= pd.DataFrame.from_dict(data)
        df= df[['c','d','h','l','o','v']]
        df['d'] = pd.to_datetime(df['d'], dayfirst= True )

        # Identify MA50
        n=50
        ma50=[] 

        for i in range(len(df["c"])-(n-1)):
            ma = round(sum(df["c"][i:i+(n)])/n,4)
            ma50.append(ma)

        df["MA50"] = pd.Series(ma50)
        df["MA50"] = df["MA50"].shift(n-1)

        # Filter uptrend price above ma50
        # Calculating the gap of both MA
        df["MAgap"] = df["c"] -  df["MA50"] 

        # Set date as index
        df.set_index('d',inplace=True,drop=False)
        df.index.name = None

        # Create signal for uptrend price above ma50 
        # Result not accurate for stochastic when just filter uptrend only
        df["Uptrend"]=df['MAgap']>0
        Uptrend=df["Uptrend"]
        Uptrend_signal=Uptrend[Uptrend==True]

        # Signal stochastic
        # Create the "L14" column in the DataFrame
        df['L14'] = df['l'].rolling(window=14).min()

        # Create the "H14" column in the DataFrame
        df['H14'] = df['h'].rolling(window=14).max()

        # Create the "%K" column in the DataFrame
        df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

        # Create the "%D" column in the DataFrame
        df['%D'] = df['%K'].rolling(window=3).mean()

        # Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
        # A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
        df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

        # Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
        # A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
        df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

        # Create a column in the DataFrame for name of stock
        df['Stock_Name'] = symbol

        # Define overbought and oversold
        overbought =df['Sell OB']
        oversold =df['Buy OS']

        # Use index(date) for create strategy using overbought and oversold
        date_os=list(oversold[oversold == True].index)
        date_ob=list(overbought[overbought == True].index)

        # Scaling certain features only
        col_names_pred = ['c', 'l', 'o', 'v','h']  
        features_pred = df[col_names_pred]

        # Scaling for prediction data
        scaler_pred = StandardScaler().fit(features_pred.values)
        features_pred = scaler_pred.transform(features_pred.values)

        # Convert the array back to a dataframe
        dataset_pred = DataFrame(features_pred)

        # Prepare prediction data that no need to scale
        col_notscale_pred=['d','Stock_Name','Buy OS']
        features_notscale_pred=df[col_notscale_pred]

        # Saving in multiple variable
        sp["{}".format(stock)] = dataset_pred
        nsp["{}".format(stock)] = features_notscale_pred


        print("----------------DONE {}------------------".format(symbol))



    # Combining all prediction data generated
    df_notscale_pred = pd.concat(list(nsp.values()),ignore_index=True)
    df_scale_pred = pd.concat(list(sp.values()),ignore_index=True)

    # Rename column
    df_scale_pred = df_scale_pred.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})


    # Concat all
    result_scaler_pred = pd.concat([df_notscale_pred, df_scale_pred], axis=1)
    result_scaler_pred = result_scaler_pred.rename(columns={'d': 'Date'})

    # Setting Dates as index
    df_all_pred = result_scaler_pred.sort_values(by="Date")
    df_all_pred.set_index('Date', inplace=True)

    # Encoding the stock names
    le = LabelEncoder()
    df_all_pred["Stock"] = le.fit_transform(df_all_pred["Stock_Name"])
    
    return df_all_pred


latest_1=prediction_data(wl)


----------------PROCESSING WILLOW------------------
----------------DONE WILLOW------------------
----------------PROCESSING LAMBO------------------
----------------DONE LAMBO------------------
----------------PROCESSING NETX------------------
----------------DONE NETX------------------
----------------PROCESSING GHLSYS------------------
----------------DONE GHLSYS------------------
----------------PROCESSING IFCAMSC------------------
----------------DONE IFCAMSC------------------
----------------PROCESSING NOVAMSC------------------
----------------DONE NOVAMSC------------------
----------------PROCESSING DIGISTA------------------
----------------DONE DIGISTA------------------
----------------PROCESSING MMAG------------------
----------------DONE MMAG------------------
----------------PROCESSING KGROUP------------------
----------------DONE KGROUP------------------
----------------PROCESSING OPENSYS------------------
----------------DONE OPENSYS------------------
----------------PROCES

----------------DONE ITRONIC------------------


In [9]:
latest = latest_1['01-01-2021':]

In [10]:
upsampled =model

In [11]:
dct_m = {}
symbol =[]
predicted=[]
date=[]

for m in range(len(latest)):
    if latest["Buy OS"].iloc[m] == True:
        # predicting the train and test set
        inputs = latest.drop(['Stock_Name','Buy OS'] ,axis=1).iloc[m].values
        
        #print(inputs)
        inputs = inputs.reshape(1,-1)
        y_pred = upsampled.predict(inputs)
        pred1 = [round(value) for value in y_pred]
        #print(pred1)
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        
        dct_m["predicted_class"] =  pred1[0]
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = latest.index[m]
        date.append(dct_m['date_uploaded'])
        print(dct_m)
    else:
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        dct_m["predicted_class"] = "No buy signal. No prediction"
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = latest.index[m]
        date.append(dct_m['date_uploaded'])
        #print("None of the stock in WL shows buy signal.\n No prediction")
        pass
        print(dct_m)
    mongo_db = {
        'Date Predicted':date,
        'Stock' :symbol,
        "Predicted_Class" :predicted,
        
    }
    Mongo_db = pd.DataFrame(mongo_db)

{'symbol': 'DPHARMA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'GRANFLO', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'MI', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'OPENSYS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'VIVOCOM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'KEYASIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'IDMENSN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'G3', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'CENSOF

{'symbol': 'EAH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'VITROX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'LAMBO', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'RGTECH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'SYSTECH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'FRONTKN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'MPAY', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'CENSOF', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-05 00:00:00')}
{'symbol': 'KESM', '

{'symbol': 'EDUSPEC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'D&O', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'JHM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'TURIYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'VIVOCOM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'JFTECH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'ELSOFT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'DNEX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-07 00:00:00')}
{'symbol': 'LAMBO', 'p

{'symbol': 'DGSB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'REVENUE', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'KRONO', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'MYEG', 'predicted_class': 0, 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'MI', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'ITRONIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'HTPADU', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'SYSTECH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-11 00:00:00')}
{'symbol': 'GRANFLO', 'predicted_class': 'No buy si

{'symbol': 'DNEX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'AEMULUS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'ELSOFT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'MMAG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'YGL', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'IFCAMSC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'RGTECH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'REVENUE', 'predicted_class': 1, 'date_uploaded': Timestamp('2021-01-12 00:00:00')}
{'symbol': 'MPI', 'predicted_class': 'No buy sign

...

{'symbol': 'GREATEC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'GTRONIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'NOVAMSC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'DPHARMA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'TRIVE', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'MI', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'DSONIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'THETA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-20 00:00:00')}
{'symbol': 'HTPADU

{'symbol': 'MIKROMB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'CENSOF', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'DGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'APPASIA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'MI', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'K1', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'AWANTEC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'NOTION', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-21 00:00:00')}
{'symbol': 'YGL', 'predi

{'symbol': 'GREATEC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'VITROX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'EDARAN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'VIS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'GRANFLO', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'NETX', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'JCY', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'MPI', 'predicted_class': 0, 'date_uploaded': Timestamp('2021-04-22 00:00:00')}
{'symbol': 'K1', 'predicted_class': 'No buy signal. No

{'symbol': 'GRANFLO', 'predicted_class': 0, 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'VSTECS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'DGSB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'DATAPRP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'PENTA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'KEYASIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-23 00:00:00')}
{'symbol': 'DNONCE', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-26 00:00:00')}
{'symbol': 'MSNIAGA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-26 00:00:00')}
{'symbol': 'TRIVE', 'predicted_class': 'No b

{'symbol': 'DPHARMA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-26 00:00:00')}
{'symbol': 'UWC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'SMTRACK', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'KRONO', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'GTRONIC', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'AEMULUS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'IDMENSN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'INARI', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-27 00:00:00')}
{'symbol': 'MI',

In [12]:
Mongo_db.set_index('Date Predicted', inplace=True)
Mongo_db.index.name = None

In [13]:
Mongo_db[Mongo_db.Predicted_Class==1]

,Stock,Predicted_Class
2021-01-04,INIX,1
2021-01-04,MSNIAGA,1
2021-01-05,EDUSPEC,1
2021-01-05,ITRONIC,1
2021-01-06,FSBM,1
...,...,...
2021-04-19,WILLOW,1
2021-04-20,AWANTEC,1
2021-04-21,DSONIC,1
2021-04-27,YGL,1


In [14]:
Mongo_db[Mongo_db.Predicted_Class==0]

,Stock,Predicted_Class
2021-01-05,SOLUTN,0
2021-01-08,HONGSENG,0
2021-01-11,INARI,0
2021-01-11,MYEG,0
2021-01-11,PENTA,0
2021-01-11,UNISEM,0
2021-01-12,APPASIA,0
2021-01-12,MICROLN,0
2021-01-12,GRANFLO,0
2021-02-04,MMSV,0
